In [3]:
!pip install langchain openai chromadb tiktoken langchain-community langchain-core

DEPRECATION: Loading egg at /home/mori/miniforge3/envs/openmm/lib/python3.12/site-packages/martini_openmm-0.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 7.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 8.0 MB/s eta 0:00:00a 0:00:01
Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [12]:
import os

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, FewShotPromptTemplate, PromptTemplate


In [13]:
assert os.environ["OPENAI_API_KEY"]

In [14]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [15]:
# 1. Basic Chat Completion
def basic_chat_completion():
    prompt = ChatPromptTemplate.from_template("以下の質問に答えてください: {question}")
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(question="日本の首都は何ですか？")
    print("Basic Chat Completion Response:", response)


In [16]:
basic_chat_completion()

Basic Chat Completion Response: 日本の首都は東京です。


In [17]:
# 2. Chain of Thought Prompting
def chain_of_thought():
    cot_prompt = PromptTemplate(
        template="""
        以下の問題を段階的に解いてください。各ステップでの考え方を説明してください。

        問題: {question}

        """,
        input_variables=["question"],
    )

    chain = LLMChain(llm=llm, prompt=cot_prompt)
    response = chain.run(
        question="1個100円のレモンを2個と1個500円のブドウを4つ買いました。\
            この時、レモンは100×2=200円、ブドウは500円×4=2000円となるため、合計で2200円となります。\
            1個100円のりんごを3個と、1個150円のみかんを2個買いました。合計金額はいくらですか？"
    )
    print("Response:", response)

In [18]:
chain_of_thought()

Response: まず、りんごの合計金額を計算します。

りんご1個あたり100円で、3個買うので、
りんごの合計金額 = 100円 × 3個 = 300円

次に、みかんの合計金額を計算します。

みかん1個あたり150円で、2個買うので、
みかんの合計金額 = 150円 × 2個 = 300円

最後に、りんごとみかんの合計金額を計算します。

合計金額 = りんごの合計金額 + みかんの合計金額
合計金額 = 300円 + 300円 = 600円

したがって、りんご3個とみかん2個を買うと合計金額は600円となります。


In [19]:
# 3. Few-Shot Prompting
def few_shot_prompting():
    # 例示用のデータ
    examples = [
        {"input": "この映画はとても面白かったです。", "output": "ポジティブ"},
        {"input": "全く期待はずれでした。", "output": "ネガティブ"},
        {"input": "まあまあでした。", "output": "ニュートラル"},
    ]

    example_prompt = PromptTemplate(
        input_variables=["input", "output"], template="入力: {input}\n出力: {output}"
    )

    few_shot_prompt = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
        prefix="以下は感情分析の例です。各テキストに対して、感情（ポジティブ/ネガティブ/ニュートラル）を判定してください。",
        suffix="入力: {input}\n出力:",
        input_variables=["input"],
    )

    chain = LLMChain(llm=llm, prompt=few_shot_prompt)
    response = chain.run(input="素晴らしい体験でした！")
    print("Few-Shot Learning Response:", response)

In [20]:
few_shot_prompting()

Few-Shot Learning Response: ポジティブ


In [ ]:
template = """
Step1 :
 
私は{input}に関連する問題を抱えています。3つの異なる解決策を考えてもらえますか？{perfect_factors}など、さまざまな要因を考慮してください。
A:
"""

prompt = PromptTemplate(input_variables=["input", "perfect_factors"], template=template)

chain1 = LLMChain(
    llm=ChatOpenAI(temperature=0, model="gpt-4o"), prompt=prompt, output_key="solutions"
)

template = """
Step 2:

提案された3つの解決策のそれぞれについて、その可能性を評価してください。\
長所と短所、初期に必要な努力、実装の難易度、潜在的な課題、および予想される結果を考慮してください。\
これらの要因に基づいて、各オプションに成功の確率と信頼レベルを割り当ててください。

{solutions}

A:"""

prompt = PromptTemplate(input_variables=["solutions"], template=template)

chain2 = LLMChain(
    llm=ChatOpenAI(temperature=0, model="gpt-4o"), prompt=prompt, output_key="review"
)

template = """
Step 3:

各解決策について、思考プロセスを深めてください。\
潜在的なシナリオ、実装戦略、必要なパートナーシップやリソース、および潜在的な障害をどのように克服するかを生成してください。\
また、予想外の結果が生じた場合の対処法も考慮してください。

{review}

A:"""

prompt = PromptTemplate(input_variables=["review"], template=template)

chain3 = LLMChain(
    llm=ChatOpenAI(temperature=0, model="gpt-4o"),
    prompt=prompt,
    output_key="deepen_thought_process",
)

template = """
Step 4:

評価とシナリオに基づいて、解決策を成功の見込みが高い順にランク付けしてください。\
各ランキングの根拠を提供し、各解決策に対する最終的な意見や考慮事項を提供してください。\
各解決策の先頭にランク（順位）を記載してください。
{deepen_thought_process}

A:"""

prompt = PromptTemplate(input_variables=["deepen_thought_process"], template=template)

chain4 = LLMChain(
    llm=ChatOpenAI(temperature=0, model="gpt-4o"),
    prompt=prompt,
    output_key="ranked_solutions",
)

In [ ]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["input", "perfect_factors"],
    output_variables=[
        "solutions",
        "review",
        "deepen_thought_process",
        "ranked_solutions",
    ],
    verbose=True,
)

In [36]:
result = overall_chain(
    {
        "input": "人類が火星に移住するための",
        "perfect_factors": "地球と火星の距離は非常に遠く、定期的な物資補給が難しい",
    }
)



> Entering new SequentialChain chain...

> Finished chain.


In [37]:
result

{'input': '人類が火星に移住するための',
 'perfect_factors': '地球と火星の距離は非常に遠く、定期的な物資補給が難しい',
 'solutions': '火星への移住に関連する問題を解決するための3つの異なる解決策を以下に提案します。\n\n1. **自給自足型の生活環境の構築**:\n   - 火星での自給自足を可能にするために、持続可能な農業技術を開発します。例えば、火星の土壌を改良して作物を育てる技術や、閉鎖型生態系を利用した水と酸素のリサイクルシステムを構築します。\n   - 3Dプリンティング技術を活用して、現地で必要な物資や部品を製造することも考えられます。これにより、地球からの物資補給の頻度を減らすことができます。\n\n2. **エネルギーの現地調達**:\n   - 太陽光発電や風力発電を利用して、火星でのエネルギーを確保します。火星の環境に適したエネルギー生成技術を開発し、エネルギーの自給自足を目指します。\n   - また、核融合技術の研究を進め、より効率的で持続可能なエネルギー源を確保することも重要です。\n\n3. **効率的な輸送システムの開発**:\n   - 地球と火星間の輸送を効率化するために、より高速で燃料効率の良い宇宙船を開発します。これにより、移動時間を短縮し、物資補給の頻度を増やすことが可能になります。\n   - また、宇宙船の再利用技術を進化させることで、コストを削減し、より頻繁な往復を可能にします。\n\nこれらの解決策は、火星移住の実現に向けた一歩となるでしょう。技術的な進歩とともに、これらのアイデアを具体化するための研究と開発が必要です。',
 'review': '以下に、提案された3つの解決策についての評価を示します。\n\n### 1. 自給自足型の生活環境の構築\n\n**長所:**\n- 火星での持続可能な生活を可能にし、地球からの依存を減らす。\n- 3Dプリンティングにより、現地での資源利用が可能になり、輸送コストを削減。\n\n**短所:**\n- 火星の過酷な環境に適応した農業技術の開発は非常に困難。\n- 閉鎖型生態系の構築には高度な技術と長期間の研究が必要。\n\n**初期に必要な努力:**\n- 大規模な研究開発と実験が必要。\n- 火星の土壌や環境に関する

In [ ]:
from pprint import pprint

pprint(result["solutions"])

('火星への移住に関連する問題を解決するための3つの異なる解決策を以下に提案します。\n'
 '\n'
 '1. **自給自足型の生活環境の構築**:\n'
 '   - '
 '火星での自給自足を可能にするために、持続可能な農業技術を開発します。例えば、火星の土壌を改良して作物を育てる技術や、閉鎖型生態系を利用した水と酸素のリサイクルシステムを構築します。\n'
 '   - '
 '3Dプリンティング技術を活用して、現地で必要な物資や部品を製造することも考えられます。これにより、地球からの物資補給の頻度を減らすことができます。\n'
 '\n'
 '2. **エネルギーの現地調達**:\n'
 '   - '
 '太陽光発電や風力発電を利用して、火星でのエネルギーを確保します。火星の環境に適したエネルギー生成技術を開発し、エネルギーの自給自足を目指します。\n'
 '   - また、核融合技術の研究を進め、より効率的で持続可能なエネルギー源を確保することも重要です。\n'
 '\n'
 '3. **効率的な輸送システムの開発**:\n'
 '   - '
 '地球と火星間の輸送を効率化するために、より高速で燃料効率の良い宇宙船を開発します。これにより、移動時間を短縮し、物資補給の頻度を増やすことが可能になります。\n'
 '   - また、宇宙船の再利用技術を進化させることで、コストを削減し、より頻繁な往復を可能にします。\n'
 '\n'
 'これらの解決策は、火星移住の実現に向けた一歩となるでしょう。技術的な進歩とともに、これらのアイデアを具体化するための研究と開発が必要です。')


In [40]:
pprint(result["review"])

('以下に、提案された3つの解決策についての評価を示します。\n'
 '\n'
 '### 1. 自給自足型の生活環境の構築\n'
 '\n'
 '**長所:**\n'
 '- 火星での持続可能な生活を可能にし、地球からの依存を減らす。\n'
 '- 3Dプリンティングにより、現地での資源利用が可能になり、輸送コストを削減。\n'
 '\n'
 '**短所:**\n'
 '- 火星の過酷な環境に適応した農業技術の開発は非常に困難。\n'
 '- 閉鎖型生態系の構築には高度な技術と長期間の研究が必要。\n'
 '\n'
 '**初期に必要な努力:**\n'
 '- 大規模な研究開発と実験が必要。\n'
 '- 火星の土壌や環境に関する詳細なデータ収集。\n'
 '\n'
 '**実装の難易度:**\n'
 '- 高い。技術的なハードルが多く、長期的な視点が必要。\n'
 '\n'
 '**潜在的な課題:**\n'
 '- 火星の土壌の毒性や放射線の影響。\n'
 '- 生態系のバランス維持の難しさ。\n'
 '\n'
 '**予想される結果:**\n'
 '- 成功すれば、火星での長期的な人類の居住が可能になる。\n'
 '\n'
 '**成功の確率と信頼レベル:**\n'
 '- 成功の確率: 中程度\n'
 '- 信頼レベル: 中程度\n'
 '\n'
 '### 2. エネルギーの現地調達\n'
 '\n'
 '**長所:**\n'
 '- エネルギーの自給自足が可能になれば、持続可能な居住が実現。\n'
 '- 太陽光や風力は火星でも利用可能な再生可能エネルギー源。\n'
 '\n'
 '**短所:**\n'
 '- 火星の環境は厳しく、太陽光や風力の効率が地球ほど高くない可能性。\n'
 '- 核融合技術はまだ実用化されていない。\n'
 '\n'
 '**初期に必要な努力:**\n'
 '- 火星環境に適したエネルギー技術の開発。\n'
 '- 核融合技術の研究と実験。\n'
 '\n'
 '**実装の難易度:**\n'
 '- 高い。特に核融合技術の実用化は非常に困難。\n'
 '\n'
 '**潜在的な課題:**\n'
 '- 火星の気象条件による発電効率の変動。\n'
 '- 核融合技術の安全性と安定性。\n'
 '\n'
 '**予想される結果:*

In [41]:
pprint(result["deepen_thought_process"])

('各解決策について、さらに詳細に考察し、実現可能性を高めるための戦略を検討します。\n'
 '\n'
 '### 1. 自給自足型の生活環境の構築\n'
 '\n'
 '**潜在的なシナリオ:**\n'
 '- 火星での農業技術が確立され、食料の自給自足が可能になる。\n'
 '- 3Dプリンティング技術を活用し、現地での建築資材や生活必需品の生産が実現。\n'
 '\n'
 '**実装戦略:**\n'
 '- 火星の土壌改良技術の開発と、耐放射線植物の遺伝子改良を進める。\n'
 '- 閉鎖型生態系の小規模モデルを地球上で構築し、シミュレーションを行う。\n'
 '- 3Dプリンティング技術の進化を促進し、現地資源を活用した製造プロセスを確立。\n'
 '\n'
 '**必要なパートナーシップやリソース:**\n'
 '- 農業技術の専門家や生態系研究者との連携。\n'
 '- 3Dプリンティング技術を持つ企業や研究機関との協力。\n'
 '- 政府や民間の資金援助。\n'
 '\n'
 '**潜在的な障害の克服方法:**\n'
 '- 火星の土壌毒性に対する耐性を持つ植物の開発。\n'
 '- 放射線シールド技術の向上。\n'
 '- 生態系のバランスを維持するためのAI技術の活用。\n'
 '\n'
 '**予想外の結果への対処法:**\n'
 '- 生態系の崩壊が起きた場合、迅速に地球からの支援を受けられる体制を整備。\n'
 '- 予期せぬ技術的問題が発生した場合、地球上でのバックアップシステムを用意。\n'
 '\n'
 '### 2. エネルギーの現地調達\n'
 '\n'
 '**潜在的なシナリオ:**\n'
 '- 太陽光発電と風力発電が火星で効率的に稼働し、エネルギーの自給自足が実現。\n'
 '- 核融合技術が実用化され、安定したエネルギー供給が可能になる。\n'
 '\n'
 '**実装戦略:**\n'
 '- 火星の気象条件に適した太陽光パネルと風力タービンの開発。\n'
 '- 核融合技術の研究を加速し、実用化に向けたプロトタイプを作成。\n'
 '\n'
 '**必要なパートナーシップやリソース:**\n'
 '- 再生可能エネルギー技術を持つ企業や研究機関との協力。\n'
 '- 核融合技術の専門家や研究機関との連携。\n'
 '-

In [43]:
pprint(result["ranked_solutions"])

('以下に、各解決策を成功の見込みが高い順にランク付けし、その根拠と考慮事項を示します。\n'
 '\n'
 '### ランク 1: エネルギーの現地調達\n'
 '\n'
 '**根拠:**\n'
 '- エネルギーはすべての活動の基盤であり、現地調達が可能になれば他のプロジェクトも進めやすくなる。\n'
 '- 太陽光発電や風力発電は既に地球上で実用化されており、火星の環境に適応させる技術開発は比較的現実的。\n'
 '- 核融合技術はまだ実用化されていないが、研究が進んでおり、長期的には安定したエネルギー供給が期待できる。\n'
 '\n'
 '**考慮事項:**\n'
 '- 火星の気象条件に適した発電技術の開発が必要。\n'
 '- 核融合技術の安全性と実用化に向けた進展を注視する必要がある。\n'
 '\n'
 '### ランク 2: 自給自足型の生活環境の構築\n'
 '\n'
 '**根拠:**\n'
 '- 食料や生活必需品の自給自足は、長期的な火星滞在に不可欠であり、持続可能な生活を支える基盤となる。\n'
 '- 3Dプリンティング技術は急速に進化しており、現地資源を活用した製造が可能になる見込みがある。\n'
 '\n'
 '**考慮事項:**\n'
 '- 火星の土壌改良や耐放射線植物の開発には時間がかかる可能性がある。\n'
 '- 生態系のバランスを維持するための技術的な課題を克服する必要がある。\n'
 '\n'
 '### ランク 3: 効率的な輸送システムの開発\n'
 '\n'
 '**根拠:**\n'
 '- 輸送システムの効率化は重要だが、技術的なハードルが高く、開発には時間と資源が必要。\n'
 '- 再利用可能な宇宙船の技術は進展しているが、完全な実用化にはさらなる研究が必要。\n'
 '\n'
 '**考慮事項:**\n'
 '- 新しい推進技術の開発には長期的な視点が必要。\n'
 '- 現在の技術を活用した代替案も検討し、柔軟な対応が求められる。\n'
 '\n'
 '### 総合的な意見\n'
 '\n'
 'エネルギーの現地調達が最も優先されるべきであり、これが実現すれば他のプロジェクトも進めやすくなります。自給自足型の生活環境の構築は、持続可能な火星生活のために重要ですが、技術的な課題が多いため、エネルギーの